# Multiple acquisition

In [1]:
import os
import sys
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
# Create 1st serial acquisition source:
acq_arduino = LDAQ.serial_communication.SerialAcquisition(port="COM3", baudrate=250000, 
                                                acquisition_name="arduino",
                                                byte_sequence=(("int16", 2), ),
                                                start_bytes=  b"\xfa\xfb",
                                                end_bytes= b"\n",
                                                sample_rate=500,
                                                channel_names=["channel 1", "channel 2"])

# Create 2nd serial acquisition source:
acq_esp = LDAQ.serial_communication.SerialAcquisition(port="COM10", baudrate=115200, 
                                          byte_sequence=(("int16", 2), ),
                                          start_bytes=  b"\xfa\xfb",
                                          end_bytes= b"\n",
                                          sample_rate=500,
                                          channel_names=["esp CH 1", "esp CH 2"],
                                          acquisition_name="esp"
                                          )

# Here the arduino and esp send out bytes over serial where a line would look like this:
# b"\xfa\xfb\x00\x01\x00\x02\n

# Explanation:
# b"\xfa\xfb" are the start bytes
# b"\n" is the end byte
# b"\x00\x01" is the first sample (int16)
# b"\x00\x02" is the second sample (int16)

In [5]:
# Create visualization:
vis = LDAQ.Visualization(refresh_rate=30)
vis.add_lines((0, 0), "esp",     [0, 1])
vis.add_lines((1, 0), "arduino", [0, 1])

vis.config_subplot((0, 0), t_span=0.5)
vis.config_subplot((1, 0), t_span=0.5)

In [11]:
# create core object and add acquisition sources:
ldaq = LDAQ.Core(acquisitions=[acq_arduino, acq_esp], visualization=vis)
# set trigger:
ldaq.set_trigger(
    source  = "arduino",
    channel = "channel 2",
    level   = 100,
    duration= 5.0)
# run acquisition:
ldaq.run(autoclose=False)

In [12]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement

{'arduino': {'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 4.994e+00, 4.996e+00,
         4.998e+00]),
  'channel_names': ['channel 1', 'channel 2'],
  'data': array([[ 512.,  576.],
         [ 512.,  639.],
         [ 512.,  700.],
         ...,
         [-512.,  384.],
         [-512.,  447.],
         [ 512.,  512.]]),
  'sample_rate': 500},
 'esp': {'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 4.994e+00, 4.996e+00,
         4.998e+00]),
  'channel_names': ['esp CH 1', 'esp CH 2'],
  'data': array([[512., 263.],
         [512., 210.],
         [512., 162.],
         ...,
         [512., 449.],
         [512., 383.],
         [512., 324.]]),
  'sample_rate': 500}}